In [1]:
import csv, os, sys
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from time import time
from datetime import timedelta,datetime
from sys import argv
import psutil as psu
from utility import *
from functions import *

In [2]:
# Initialize a spark session.
@metrics
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [3]:
spark = init_spark()

--------- BEFORE CALL TO FUNCTION ---------


 ========== SYSTEM INFO ===========

Memory in use: 5.14GiB
Disk in use: 71.70%
Disk free: 4.11GiB
Time on CPU: 4:04:10.400000
CPU in use: 14.50%


 --------- AFTER CALL TO FUNCTION ---------


 ========== SYSTEM INFO ===========

Memory in use: 5.12GiB
Disk in use: 71.70%
Disk free: 4.11GiB
Time on CPU: 4:04:23.410000
CPU in use: 36.50%

----------> Execution Time: 3.94852 seconds


# Load data from file into spark

In [10]:
filename_train = "../dataset/train.csv"
filename_test = "../dataset/valid.csv"

train_rdd = spark.read.csv(filename_train, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
test_rdd = spark.read.csv(filename_test, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
train_rdd.show(5)

+--------+--------------------+--------------------+--------------------+-------------------+--------+
|      Id|               Title|                Body|                Tags|       CreationDate|       Y|
+--------+--------------------+--------------------+--------------------+-------------------+--------+
|34552656|Java: Repeat Task...|<p>I'm already fa...|      <java><repeat>|2016-01-01 00:21:59|LQ_CLOSE|
|34553034|Why are Java Opti...|<p>I'd like to un...|    <java><optional>|2016-01-01 02:03:20|      HQ|
|34553174|Text Overlay Imag...|<p>I am attemptin...|<javascript><imag...|2016-01-01 02:48:24|      HQ|
|34553318|Why ternary opera...|<p>The question i...|<swift><operators...|2016-01-01 03:30:17|      HQ|
|34553755|hide/show fab wit...|<p>I'm using cust...|<android><materia...|2016-01-01 05:21:48|      HQ|
+--------+--------------------+--------------------+--------------------+-------------------+--------+
only showing top 5 rows



In [11]:
training = train_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"])# change to collect()

testing = test_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"]) # change to collect()

### Associated functions

In [7]:
@metrics
def load_train_test_rdd(filename_train, filename_test):
    train_rdd = spark.read.csv(filename_train, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
    test_rdd = spark.read.csv(filename_test, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
    return train_rdd, test_rdd
    

In [8]:
@metrics
def transform_rdd_to_df(train_rdd,test_rdd):
    training = train_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"])

    testing = test_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"])
    
    return training, testing
    

In [22]:
filename_train = "../dataset/train.csv"
filename_test = "../dataset/valid.csv"
train_rdd, test_rdd = load_train_test_rdd(filename_train, filename_test)
train_df, test_df = transform_rdd_to_df(train_rdd,test_rdd)

# Prepare pipeline

In [20]:
def get_stop_word_remover(input_col_name, stopwords):
    return StopWordsRemover(inputCol=input_col_name, outputCol="filtered").setStopWords(stopwords)
    

In [9]:
@metrics
def get_heuristics(stop_word_file):
    # HEURISTIC 1 - Tokenize the words
    regexTokenizer = RegexTokenizer(inputCol="Question", outputCol="words", pattern="\\W")
    
    # HEURISTIC 2 - Remove the stopwords
    stop_words = []
    with open(stop_word_file, "r") as text_file:
        stop_words = text_file.read().split('\n')
    
    stopwordsRemover = get_stop_word_remover("words", stop_words)
    
    return regexTokenizer, stopwordsRemover
       

In [28]:
stop_word_file = "../dataset/stop_words.txt"
regexTokenizer, stopwordsRemover = get_heuristics(stop_word_file)

In [23]:
@metrics
def get_bag_of_word_model(features_col_name, label_col_name):
    countVectors = CountVectorizer(inputCol=features_col_name, outputCol="features")
    indexed_features = StringIndexer(inputCol = label_col_name, outputCol = "label")
    return countVectors, indexed_features
     

In [54]:
countVectors_h1, indexed_features_h1 = get_bag_of_word_model("words", "Output")
countVectors_h2, indexed_features_h2 = get_bag_of_word_model("filtered", "Output")

BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 7.7%
Uptime: 19:21:47.528509
Disk in use: 65.5%
Memory in use: 6.625556945800781GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:47.660760
Disk in use: 65.5%
Memory in use: 6.624839782714844GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000
Execution Time: 0.03131103515625
BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:47.761858
Disk in use: 65.5%
Memory in use: 6.624351501464844GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:47.871834
Disk in use: 65.5%
Memory in use: 6.623928070068359GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000
Execution Time: 0.009263992309570312


In [10]:
@metrics
def get_pipeline(*args):
    return Pipeline(stages=[*args])

In [56]:
pipeline_h1 = get_pipeline(regexTokenizer, stopwordsRemover, countVectors_h2, indexed_features_h2)

BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 6.3%
Uptime: 19:21:52.424083
Disk in use: 65.5%
Memory in use: 6.653079986572266GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.890000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 4.9%
Uptime: 19:21:52.527166
Disk in use: 65.5%
Memory in use: 6.653156280517578GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.940000
Execution Time: 0.0004260540008544922
BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 4.8%
Uptime: 19:21:52.628059
Disk in use: 65.5%
Memory in use: 6.653175354003906GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.980000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:52.734210
Disk in use: 65.5%
Memory in use: 6.653194427490234GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.980000
Execution Time: 0.0003941059112548828


# Process data through Pipeline train & test

In [11]:
@metrics
def get_pipeline_model(pipeline, data):
    """ We should use the same pipeline model on training and testing """
    return pipeline.fit(data)


In [12]:
@metrics
def transform_data_through_pipeline(model, data):
    data_transformed = model.transform(data)
    return data_transformed

In [21]:
model_pipeline = get_pipeline_model(pipeline_h1, training)
data_h1_train = transform_data_through_pipeline(model_pipeline, train_df)
data_h1_test = transform_data_through_pipeline(model_pipeline, test_df)
data_h1_train.show()

Time: 16.91487216949463
+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|Java: Repeat Task...|LQ_CLOSE|[java, repeat, ta...|(201488,[0,1,2,3,...|  1.0|
|Why are Java Opti...|      HQ|[why, are, java, ...|(201488,[0,1,4,7,...|  0.0|
|Text Overlay Imag...|      HQ|[text, overlay, i...|(201488,[0,1,2,3,...|  0.0|
|Why ternary opera...|      HQ|[why, ternary, op...|(201488,[0,1,2,3,...|  0.0|
|hide/show fab wit...|      HQ|[hide, show, fab,...|(201488,[0,1,4,5,...|  0.0|
|Accessing pointer...|LQ_CLOSE|[accessing, point...|(201488,[0,1,2,3,...|  1.0|
|How To Disable 2n...| LQ_EDIT|[how, to, disable...|(201488,[1,4,8,15...|  2.0|
|Resizing containe...| LQ_EDIT|[resizing, contai...|(201488,[1,2,3,4,...|  2.0|
|Changing Theme in...|      HQ|[changing, theme,...|(201488,[0,1,2,3,...|  0.0|
|TextBox Value D

**Vocabulary**

In [61]:
# vocabulary frequency without using the countVectorizer helper column that we generated
counts = data_h1_train.select(f.explode('words').alias('col')).groupBy('col').count().take(20)
print({row['col']: row['count'] for row in counts})

{'1293400': 1, 'mypass': 5, 'connected': 433, 'few': 902, 'input': 9663, 'online': 599, '389999': 1, 'travel': 74, '836400': 1, 'those': 1212, 'still': 2194, 'thread1': 26, 'hope': 513, 'recognize': 120, 'parentheses': 89, 'arguments': 696, 'persist': 74, '2bxhsys2c47eyjfhpmroalpxz5suigeubqu7hjuvfvwpoa0xri3iljvhq5qgbwtwpe1x0': 2, 'pabu': 1, 'some': 8532}


In [62]:
# using the countVectorizer helper column that we generated
# https://stackoverflow.com/questions/50255356/pyspark-countvectorizer-and-word-frequency-in-a-corpus
# counts = data_h1_train.select('words').take(20)
# print(dict(zip(vocabulary, counts[0]['words'].values)))

NameError: name 'model' is not defined

In [73]:
data_h1_test = process_data_in_pipeline(pipeline_h1, testing)
data_h1_test.show()

+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|How to get all th...| LQ_EDIT|[how, to, get, al...|(86590,[1,2,4,6,9...|  2.0|
|Retrieve all exce...| LQ_EDIT|[retrieve, all, e...|(86590,[1,2,7,9,1...|  2.0|
|Pandas: read_html...|      HQ|[pandas, read_htm...|(86590,[0,1,2,3,4...|  0.0|
|Reader Always gim...| LQ_EDIT|[reader, always, ...|(86590,[1,2,4,5,7...|  2.0|
|php rearrange arr...| LQ_EDIT|[php, rearrange, ...|(86590,[1,2,4,5,6...|  2.0|
|How do I make a c...|LQ_CLOSE|[how, do, i, make...|(86590,[0,1,2,3,4...|  1.0|
|how can i create ...| LQ_EDIT|[how, can, i, cre...|(86590,[1,5,6,9,1...|  2.0|
|Re-exporting ES6 ...|      HQ|[re, exporting, e...|(86590,[0,1,2,3,4...|  0.0|
|Fetch API with Co...|      HQ|[fetch, api, with...|(86590,[0,1,2,4,5...|  0.0|
|Print list conten...|LQ_CLOSE|[print, l

In [13]:
data_h1_test  = model_pipeline.transform(testing)
data_h1_test.show()

+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|How to get all th...| LQ_EDIT|[how, to, get, al...|(201488,[1,2,4,6,...|  2.0|
|Retrieve all exce...| LQ_EDIT|[retrieve, all, e...|(201488,[1,2,7,8,...|  2.0|
|Pandas: read_html...|      HQ|[pandas, read_htm...|(201488,[0,1,2,3,...|  0.0|
|Reader Always gim...| LQ_EDIT|[reader, always, ...|(201488,[1,2,4,5,...|  2.0|
|php rearrange arr...| LQ_EDIT|[php, rearrange, ...|(201488,[1,2,4,5,...|  2.0|
|How do I make a c...|LQ_CLOSE|[how, do, i, make...|(201488,[0,1,2,3,...|  1.0|
|how can i create ...| LQ_EDIT|[how, can, i, cre...|(201488,[1,5,6,8,...|  2.0|
|Re-exporting ES6 ...|      HQ|[re, exporting, e...|(201488,[0,1,2,3,...|  0.0|
|Fetch API with Co...|      HQ|[fetch, api, with...|(201488,[0,1,2,4,...|  0.0|
|Print list conten...|LQ_CLOSE|[print, l

# Split datasets

In [13]:
@metrics
def split_dataset(data, distribution):
    return data.randomSplit([distribution, 1-distribution], seed = 1234)
    

In [15]:
train_h1, validate_h1 = split_dataset(data_h1_train, 0.7)

# Hyperparameter tuning | is this doing anything? we are not using any data

In [14]:
@metrics
def get_best_smoothing_values(target_col, prediction_col):
    # Create grid to find best smoothing
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
    paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
    cvEvaluator = MulticlassClassificationEvaluator(labelCol=target_col, predictionCol=prediction_col)

    # Cross-validate all smoothing values
    cv = CrossValidator(estimator=nb, estimatorParamMaps=paramGrid, evaluator=cvEvaluator)
    return cv
    

In [17]:
cv = get_best_smoothing_values("label", "prediction")

# Train Model

In [15]:
@metrics
def train_naive_bayes_model(cv, data):
    model = cv.fit(data)
    return model

In [18]:
nb_model = train_naive_bayes_model(cv,train)

# Predict

In [16]:
@metrics
def predict(model, data):
    predictions = model.transform(data)
    return predictions

In [19]:
predictions = predict(nb_model, test)

In [20]:
type(cvPredictions)

pyspark.sql.dataframe.DataFrame

In [21]:
cvPredictions.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  2.0|       1.0|
|  1.0|       1.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [22]:
cvPredictions_test = cvModel.transform(data_h1_test)

In [23]:
cvPredictions_test.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  2.0|       2.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  2.0|       2.0|
|  2.0|       2.0|
|  1.0|       1.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  2.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  2.0|       2.0|
|  2.0|       2.0|
+-----+----------+
only showing top 20 rows



# Evaluate Performance of model

In [17]:
@metrics
def evaluate_model(target_col, prediction_col, predictionAndTarget):
    evaluatorMulti = MulticlassClassificationEvaluator(labelCol=target_col, predictionCol=prediction_col)
    # Get metrics
    acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
    f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
    weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
    weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
    print("\n******** Metrics **********")
    print ("Model Accuracy: {:.3f}%".format(acc*100))
    print ("Model f1-score: {:.3f}%".format(f1*100))
    print ("Model weightedPrecision: {:.3f}%".format(weightedPrecision*100))
    print ("Model weightedRecall: {:.3f}%".format(weightedRecall*100))

In [26]:
evaluate_model("label", "prediction", cvPredictions.select("label","prediction"))

Model Accuracy:  0.7653008490217793
Model f1-score:  0.7675841714238029
Model weightedPrecision:  0.7901628512971111
Model weightedRecall:  0.7653008490217792


In [27]:
# testing dataset
evaluate_model("label", "prediction", cvPredictions_test.select("label","prediction"))

Model Accuracy:  0.7633333333333333
Model f1-score:  0.7649182102284617
Model weightedPrecision:  0.7866094786891012
Model weightedRecall:  0.7633333333333333


# Creating one big iteration for all the pipelines

In [4]:
def main_spark():
    print("##########################################")
    print("############# Start Spark ################")
    print("##########################################")
    global spark
    spark = init_spark()
    filename_train = "../dataset/train.csv"
    filename_test = "../dataset/valid.csv"
    stop_word_file = "../dataset/stop_words.txt"
    print("\n#########################################")
    print("####### Load dataset in spark rdd #######")
    print("###########################################")
    train_rdd, test_rdd = load_train_test_rdd(filename_train, filename_test)
    print("\n#########################################")
    print("########## Transform rdd to df ############")
    print("##########################################")
    train_df, test_df = transform_rdd_to_df(train_rdd,test_rdd)
    print("\n##########################################")
    print("########## Create Heuristics #############")
    print("##########################################")
    regexTokenizer, stopwordsRemover = get_heuristics(stop_word_file)
    countVectors_h1, indexed_features_h1 = get_bag_of_word_model("words", "Output")
    countVectors_h2, indexed_features_h2 = get_bag_of_word_model("filtered", "Output")
    print("\n##########################################")
    print("############ Construct pipeline ############")
    print("############################################")
    pipeline = get_pipeline(regexTokenizer, stopwordsRemover, countVectors_h2, indexed_features_h2)
    print("\n##########################################")
    print("########### Train pipeline model ###########")
    print("############################################")
    model_pipeline = get_pipeline_model(pipeline, train_df)
    print("\n#############################################")
    print("####### Transform train data through pipeline #####")
    print("################################################")
    train = transform_data_through_pipeline(model_pipeline, train_df)
    print("\n##################################################")
    print("####### Transform test data through pipeline #######")
    print("##################################################")
    test = transform_data_through_pipeline(model_pipeline, test_df)
    print("\n###################################################")
    print("####### Train naive base classifier model #######")
    print("####################################################")
    cv = get_best_smoothing_values("label", "prediction")
    nb_model = train_naive_bayes_model(cv,train)
    print("\n##############################################################")
    print("### Predict test data using naive base classifier model ######")
    print("################################################################")
    predictions = predict(nb_model, test)
    print("####################################")
    print("####### Evaluate predictions #######")
    print("#####################################")
    evaluate_model("label", "prediction",predictions.select("label","prediction"))

# Performance Analysis

In [5]:
# decorator
def metrics(fun):
    def wrapper(*args, **kwargs):
        print('--------- BEFORE CALL TO FUNCTION ---------')
        fun_name = fun.__name__
        info_before_call = system_info()
        start = time()
        rv = fun(*args, **kwargs)
        duration = time() - start
        print('\n\n --------- AFTER CALL TO FUNCTION ---------')
        info_after_call = system_info()
        system_dict = {fun_name: [info_before_call, info_after_call, duration]}

        print("\n----------> Execution Time: {:.5f} seconds".format(duration))

        with open("system_info.json", "r+") as file:
            data = json.load(file)
            data.update(system_dict)
            file.seek(0)
            json.dump(data, file, default=str)
        return rv

    return wrapper

In [4]:
def system_info():
    info = {
        "CPU in use": "{:.2f}%".format(psu.cpu_percent(interval=.1)),
        "Time on CPU": timedelta(seconds=psu.cpu_times().system + psu.cpu_times().user),
        "Memory in use": "{:.2f}GiB".format(psu.virtual_memory().available / (1024 ** 3)),
        "Disk in use": "{:.2f}%".format(psu.disk_usage('/').percent),
        "Disk free": "{:.2f}GiB".format(psu.disk_usage('/').free / (1024 ** 3)),
    }

    print("\n\n ========== SYSTEM INFO ===========\n\n" + "\n".join(
        ["{}: {}".format(key, value) for key, value in info.items()]))

    return info

In [26]:
system_info()



 SYSTEMINFO

CPU in use: 15.0%
Uptime: 19:15:26.625819
Disk in use: 65.5%
Memory in use: 6.625919342041016MiB
Dsik free: 5.499961853027344MiB
Time on CPU: 7:32:57.350000


In [25]:
print(timedelta(seconds=time()-psu.boot_time()))

19:15:19.796094


In [24]:
seconds=time()-psu.boot_time()

In [62]:
import multiprocessing
multiprocessing.cpu_count()

8

In [3]:
def process_info():
    processes_info = []
    for process in psu.process_iter(attrs=(
            'name', 'cmdline', 'pid', 'create_time', 'cpu_percent', 'cpu_times', 'num_threads', 'memory_percent')):

        if process.info["name"] is not None and ("python3.5" in process.info["name"] or "java" in process.info["name"]):
            mem = process.info['memory_percent']
            info = {
                "PID": process.info["pid"],
                "Process name": process.info["name"],
                "Current time": datetime.fromtimestamp(time()).strftime("%Y-%m-%d %H:%M:%S"),
                "Create time": datetime.fromtimestamp(process.create_time()).strftime("%Y-%m-%d %H:%M:%S"),
                "Uptime": timedelta(seconds=time() - process.info["create_time"]),
                "CPU in use": "{:.2f}%".format(process.info['cpu_percent']),
                "Time on CPU": timedelta(seconds=process.info["cpu_times"].system + process.info["cpu_times"].user),
                "Nb of threads": process.info["num_threads"],
                "Memory in use": "{:.2f}%".format(mem),
                "Memory_usage": "{:.2f} GiB".format(psu.virtual_memory().total * (mem / 100) / (1024 ** 3)),
            }
            processes_info.append(info)
            print("\n\n ********* PROCESS INFO *********\n\n" + "\n".join(
                ["{}: {}".format(key, value) for key, value in info.items()]))

    process_dict = {"processes": processes_info}

    with open('processes_info.json', 'w') as fp:
        json.dump(process_dict, fp, default=str)

    return processes_info           

In [29]:
process_info()



 PROCESS INFO

Uptime: 12:13:20.380172
Create time: 2021-04-10 23:46:55
Nb of threads: 4
Memory_usage: 0.03 GiB
Memory in use: 0.16%
CPU in use: 0.20%
PID: 5065
Time on CPU: 0:00:06.343331


 PROCESS INFO

Uptime: 12:13:05.514102
Create time: 2021-04-10 23:47:10
Nb of threads: 10
Memory_usage: 0.03 GiB
Memory in use: 0.18%
CPU in use: 0.20%
PID: 5078
Time on CPU: 0:00:03.614826


 PROCESS INFO

Uptime: 12:12:23.593419
Create time: 2021-04-10 23:47:52
Nb of threads: 1
Memory_usage: 0.00 GiB
Memory in use: 0.00%
CPU in use: 0.00%
PID: 5095
Time on CPU: 0:00:00.970705
